# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 12:03PM,243876,19,TR-RG-08152022,"GCH Granules USA, Inc.",Released
1,Aug 10 2022 11:51AM,243874,19,ACG-2102490902,ACG North America LLC,Released
2,Aug 10 2022 11:51AM,243874,19,ACG-2102490901,ACG North America LLC,Released
3,Aug 10 2022 11:43AM,243872,10,0085932313,ISDIN Corporation,Released
4,Aug 10 2022 11:43AM,243872,10,0085932321,ISDIN Corporation,Released
5,Aug 10 2022 11:43AM,243872,10,0085932334,ISDIN Corporation,Released
6,Aug 10 2022 11:43AM,243872,10,0085932337,ISDIN Corporation,Released
7,Aug 10 2022 11:43AM,243872,10,0085932336,ISDIN Corporation,Released
8,Aug 10 2022 11:43AM,243872,10,0085932344,ISDIN Corporation,Released
9,Aug 10 2022 11:43AM,243872,10,0085932343,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,243870,Executing,1
35,243872,Released,21
36,243873,Released,1
37,243874,Released,2
38,243876,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243870,NaN,1.0,NaN
243872,NaN,NaN,21.0
243873,NaN,NaN,1.0
243874,NaN,NaN,2.0
243876,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0.0,0.0,9.0
243778,22.0,6.0,1.0
243795,14.0,6.0,0.0
243805,0.0,0.0,15.0
243808,24.0,16.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0,0,9
243778,22,6,1
243795,14,6,0
243805,0,0,15
243808,24,16,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,0,0,9
1,243778,22,6,1
2,243795,14,6,0
3,243805,0,0,15
4,243808,24,16,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,,,9
1,243778,22,6,1
2,243795,14,6,
3,243805,,,15
4,243808,24,16,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc."
1,Aug 10 2022 11:51AM,243874,19,ACG North America LLC
3,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation
24,Aug 10 2022 11:36AM,243873,19,ACG North America LLC
25,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc."
37,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc."
38,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC"
41,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.
61,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC"
66,Aug 10 2022 10:19AM,243865,16,American International Chemical


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",,,1
1,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,,,2
2,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,,,21
3,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,,,1
4,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",,9,3
5,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",,1,
6,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",,,3
7,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,,11,9
8,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC",,,5
9,Aug 10 2022 10:19AM,243865,16,American International Chemical,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1,,
1,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2,,
2,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21,,
3,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,1,,
4,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",3,9,
5,Aug 10 2022 10:34AM,243870,10,"Methapharm, Inc.",,1,
6,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",3,,
7,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,9,11,
8,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC",5,,
9,Aug 10 2022 10:19AM,243865,16,American International Chemical,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1,,
1,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2,,
2,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21,,
3,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,1,,
4,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",3,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2.0,NaN,NaN
2,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21.0,NaN,NaN
3,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,1.0,NaN,NaN
4,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",3.0,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2.0,0.0,0.0
2,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21.0,0.0,0.0
3,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,1.0,0.0,0.0
4,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",3.0,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1463102,44.0,17.0,2.0
102,243818,1.0,0.0,0.0
15,975449,45.0,20.0,0.0
16,243865,1.0,0.0,0.0
19,1463112,6.0,21.0,14.0
20,1706696,64.0,31.0,46.0
21,487658,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1463102,44.0,17.0,2.0
1,102,243818,1.0,0.0,0.0
2,15,975449,45.0,20.0,0.0
3,16,243865,1.0,0.0,0.0
4,19,1463112,6.0,21.0,14.0
5,20,1706696,64.0,31.0,46.0
6,21,487658,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,44.0,17.0,2.0
1,102,1.0,0.0,0.0
2,15,45.0,20.0,0.0
3,16,1.0,0.0,0.0
4,19,6.0,21.0,14.0
5,20,64.0,31.0,46.0
6,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,102,Released,1.0
2,15,Released,45.0
3,16,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20,21
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,14.0,46.0,0.0
Executing,17.0,0.0,20.0,0.0,21.0,31.0,2.0
Released,44.0,1.0,45.0,1.0,6.0,64.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,14.0,46.0,0.0
1,Executing,17.0,0.0,20.0,0.0,21.0,31.0,2.0
2,Released,44.0,1.0,45.0,1.0,6.0,64.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,14.0,46.0,0.0
1,Executing,17.0,0.0,20.0,0.0,21.0,31.0,2.0
2,Released,44.0,1.0,45.0,1.0,6.0,64.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()